In [ ]:
#!apt-get update
#!apt-get install -y swig python3-dev

In [ ]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip install -r requirements.txt

In [ ]:
# %% [markdown]
# # PPO en CarRacing-v3 con TorchRL (gráficas + tqdm)
#
# Este Notebook enseña cómo usar PPO de TorchRL para entrenar CarRacing-v3 (con acciones discretas), e incluye gráficos en vivo (como en el IPYNB que enviaste) y una barra de progreso (`tqdm`) que muestra el avance por iteración, junto con “Avg Reward (Train)” y “Avg Reward (Eval)”.
#
# **Dependencias**:
# ```
# pip install torch torchvision torchrl gymnasium[box2d] tqdm matplotlib
# ```
#
# ———————————————————————————————————————————————————————————————————————————————

# %% [markdown]
# ## 1. Importaciones y utilidades de graficado
#
# — En esta sección importamos todo lo necesario: Torch, TorchRL, Gym, Matplotlib, tqdm, y definimos la función `plot(logs)` (idéntica a la de tu notebook) que reconstruye la figura en cada iteración.

# %% [code]
import warnings
warnings.filterwarnings("ignore")
import os
import uuid

import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gymnasium as gym
import numpy as np

from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from tqdm import tqdm

# TorchRL
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.transforms import ToTensorImage, Compose
from torchrl.collectors import SyncDataCollector
from torchrl.envs.utils import ExplorationType, set_exploration_type

from torchrl.modules import SafeModule, ProbabilisticActor, ValueOperator, TanhNormal
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE

# — Función de graficado (idéntica a la del IPYNB de ejemplo) —
def plot(logs):
    """
    Recibe un dict `logs` con listas en:
      logs["train_reward"] = [r1, r2, r3, ...]
      logs["eval_reward"]  = [e1, e2, e3, ...]
    y dibuja dos subplots: 
      · Avg Reward (Train)
      · Avg Reward (Eval)
    Llamar esta función tras actualizar `logs` para redibujar la gráfica en línea.
    """
    clear_output(wait=True)
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes = axes.flatten()
    titles = ["Avg Reward (Train)", "Avg Reward (Eval)"]
    data = [
        (logs["train_reward"], "blue"),
        (logs["eval_reward"],  "green"),
    ]
    for ax, title, (y, color) in zip(axes, titles, data):
        ax.plot(y, color=color)
        ax.set_title(title)
        ax.relim()
        ax.autoscale_view()
    plt.tight_layout()
    display(fig)
    plt.close(fig)

# %% [markdown]
# ## 2. Wrapper discreto para CarRacing-v3
#
# El entorno original de CarRacing usa un espacio continuo de 3 dimensión (`[steer, gas, brake]`), pero queremos usar PPO discreto con 5 acciones:
#
# 1. `0`: no-op            → `[ 0.0,  0.0,  0.0 ]`  
# 2. `1`: acelerar        → `[ 0.0,  1.0,  0.0 ]`  
# 3. `2`: frenar          → `[ 0.0,  0.0,  0.8 ]`  
# 4. `3`: girar izquierda → `[−1.0,  0.0,  0.0 ]`  
# 5. `4`: girar derecha   → `[ 1.0,  0.0,  0.0 ]`
#
# Este wrapper convierte un entero discreto (0–4) en el vector contínuo que CarRacing espera.

# %% [code]
class DiscreteCarRacingWrapper(gym.ActionWrapper):
    """
    Convierte la acción discreta en un vector [steer, gas, brake].
    """
    def __init__(self, env: gym.Env):
        super().__init__(env)
        self.discrete_actions = [
            np.array([ 0.0,  0.0,  0.0 ], dtype=np.float32),  # 0: no-op
            np.array([ 0.0,  1.0,  0.0 ], dtype=np.float32),  # 1: acelerar
            np.array([ 0.0,  0.0,  0.8 ], dtype=np.float32),  # 2: frenar
            np.array([-1.0,  0.0,  0.0 ], dtype=np.float32),  # 3: girar izquierda
            np.array([ 1.0,  0.0,  0.0 ], dtype=np.float32),  # 4: girar derecha
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))

    def action(self, action_discrete: int) -> np.ndarray:
        return self.discrete_actions[int(action_discrete)]

# %% [markdown]
# ## 3. Crear entornos TorchRL
#
# Para PPO necesitamos:
# 1. **entorno de entrenamiento** (sin render, con transform).  
# 2. **entorno de evaluación** (también sin render, pero lo usaremos solo para medir rendimiento).  
#
# Usamos `GymEnv` con `from_pixels=True` + `ToTensorImage()` para que cada paso regrese un Tensordict con:
# - `"pixels"`: Tensor `[3×96×96]` en `[0,1]`.  
# - `"action"`: int escalar en `[0..4]`.  

# %% [code]
def make_torchrl_env(seed: int = 0, device: str = "cpu"):
    # 1) Gym base
    gym_env = gym.make("CarRacing-v3", render_mode=None)
    # 2) Wrapper discreto
    gym_env = DiscreteCarRacingWrapper(gym_env)
    gym_env.seed(seed)
    # 3) GymEnv (TorchRL) con frames en Tensor
    trl_env = GymEnv(
        env=gym_env,
        from_pixels=True,
        pixels_only=True,
        device=device
    )
    # 4) Transform (uint8 HWC → float32 CHW en [0,1])
    trl_env.set_transform(Compose(ToTensorImage()))
    return trl_env

# %% [markdown]
# ## 4. Definir Actor-Crítico con SafeModule + ProbabilisticActor + ValueOperator
#
# - La red convolucional es idéntica a la del ejemplo: 3 → 32 conv8×8/stride 4 → 64 conv4×4/stride 2 → 64 conv3×3/stride 1 → Flatten → FC 512.  
# - **Policy head**: saca logits sobre 5 acciones discretas.  
# - **Value head**: saca un escalar (valor del estado).  
#
# Con TorchRL envolvemos así:
# 1. `SafeModule` para el feature extractor y dos cabezas separadas (`policy_head`, `value_head`).  
# 2. `ProbabilisticActor` para convertir “logits” en una `torch.distributions.Categorical`.  
# 3. `ValueOperator` para exponer la salida de valor.  

# %% [code]
# 4.1) Red CNN base
class CNNBody(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)   # → [32,23,23]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)  # → [64,10,10]
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)  # → [64,8,8]
        self.flatten_dim = 64 * 8 * 8
        self.fc = nn.Linear(self.flatten_dim, 512)

    def forward(self, obs_image: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(obs_image))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.flatten_dim)
        x = F.relu(self.fc(x))
        return x  # → [B,512]

# 4.2) Construcción del módulo completo con dos salidas
class ActorCriticNet(nn.Module):
    def __init__(self, num_actions: int = 5):
        super().__init__()
        self.body = CNNBody()
        # Cabeza de política: produce logits [B,5]
        self.policy_head = nn.Linear(512, num_actions)
        # Cabeza de valor: produce valor escalar [B,1]
        self.value_head  = nn.Linear(512, 1)

    def forward(self, obs_image: torch.Tensor):
        features = self.body(obs_image)                # [B,512]
        logits   = self.policy_head(features)          # [B,5]
        value    = self.value_head(features).squeeze(-1)  # [B]
        return logits, value

# 4.3) Armado de SafeModule + ProbabilisticActor + ValueOperator
def make_ac_modules(device: str = "cpu"):
    """
    Devuelve:
      - actor_module: ProbabilisticActor que espera 'pixels' y saca 'action', 'log_prob'
      - value_module: ValueOperator que espera 'pixels' y saca 'value'
    """
    base_net = ActorCriticNet(num_actions=5).to(device)
    # Para la política:
    actor_sm = SafeModule(
        base_net, 
        in_keys=["pixels"],            # recibe tensor 'pixels'
        out_keys=["logits", "value"],  # reuse: esta forward devuelve (logits, value)
        function=lambda m, x: m(x)     # m(x) → (logits,value)
    )
    # De los logits salimos con Categorical:
    actor_module = ProbabilisticActor(
        actor_sm,
        in_keys=["logits"],
        out_keys=["action", "log_prob"],
        distribution_class=Categorical,
        distribution_kwargs={},       # Categorical(logits=logits)
        return_log_prob=True,
        default_interaction_mode="random",  # en train samplea, en eval podemos hacer mode="mode"
    )

    # Para la cabeza de valor:
    value_sm = SafeModule(
        base_net, 
        in_keys=["pixels"],
        out_keys=["value"], 
        function=lambda m, x: m(x)[1].unsqueeze(-1)  
        # m(x)[1] ya es [B], le agrego dim -1 para que ValueOperator espere [B,1]
    )
    value_module = ValueOperator(
        in_keys=["value"],   # recibe 'value'
        out_keys=["state_value"],
    )

    return actor_module, value_module

# %% [markdown]
# ## 5. Configuración del Collector y pérdida PPO (ClipPPOLoss + GAE)
#
# - `frames_per_batch`: cuántos pasos (frames) recolectamos antes de hacer update de PPO.  
# - `num_iterations`: cuántas veces repetimos ese ciclo recolectar→optimizar.  
# - Internamente, ClipPPOLoss calculará la pérdida de crítica (MSE), la pérdida de política con clipping, y entropía.  
# - Para GAE definimos un objeto `GAE(gamma, lam)` para generar las ventajas.  

# %% [code]
# 5.1) Hiperparámetros
frames_per_batch   = 1024    # pasos a recolectar por iteración
num_iterations     = 256     # cuántas iteraciones (epocas) de PPO
ppo_epoch          = 4       # cuántas pasadas sobre cada batch de datos
sub_batch_size     = 128     # minibatch para optimizador
gamma              = 0.99
lam                = 0.95
learning_rate      = 2e-4

# 5.2) Crear entornos
device = "cuda" if torch.cuda.is_available() else "cpu"
env_train = make_torchrl_env(seed=123, device=device)
env_eval  = make_torchrl_env(seed=456, device=device)

# 5.3) Crear módulos Actor y Critic
actor_module, value_module = make_ac_modules(device=device)

# 5.4) Establecer exploración por defecto (importante para PPO “on-policy”)
#     TorchRL usa ExplorationType.RANDOM en training y EXPLORATION.EVAL en evaluación.
set_exploration_type(ExplorationType.RANDOM)

# 5.5) ClipPPOLoss y GAE
#    - td_steps=frames_per_batch: cuántos pasos en el rollout
#    - clip_eps=0.2: epsilon de clipping
#    - value_loss_coeff=0.5, entropy_coeff=0.01  (valores típicos)
clip_ppo_loss = ClipPPOLoss(
    actor=actor_module,
    value_network=value_module,
    clip_eps=0.2,
    value_loss_coeff=0.5,
    entropy_coeff=0.01,
    gamma=gamma,
    lambda_=lam,
    max_grad_norm=0.5,
)

# 5.6) Optimizadores (uno solo que abarque params de actor y critic)
optimizer = torch.optim.Adam(
    list(actor_module.parameters()) + list(value_module.parameters()),
    lr=learning_rate
)

# 5.7) GAE (para calcular ventajas dentro de ClipPPOLoss)
gae_module = GAE(
    gamma=gamma,
    lambda_=lam,
    reduction="mean"
)

# 5.8) Collector síncrono (auto‐reset) para recolección de transiciones
from torchrl.collectors import SyncDataCollector
from torchrl.envs import AutoResetEnv

# AutoResetEnv se encarga de llamar reset() internamente
train_collector = SyncDataCollector(
    env=AutoResetEnv(env_train),
    policy=actor_module,
    frames_per_batch=frames_per_batch,
    total_frames=None,
    device=device,
)

# ———————————————————————————————————————————————————————————————————————————————

# %% [markdown]
# ## 6. Rutina de entrenamiento + evaluación con tqdm y plot(logs)
#
# En cada iteración (de `0` a `num_iterations-1`):
# 1. Recolectar `frames_per_batch` pasos con `train_collector`.  
# 2. Obtener un Tensordict con las transiciones:  
#    - `"pixels"`, `"action"`, `"log_prob"`, `"next_pixels"`, `"reward"`, `"done"`, `"state_value"`, etc.  
# 3. Calcular ventajas y retornos con `gae_module`.  
# 4. Llamar a `clip_ppo_loss` sobre ese Tensordict para optimizar (usa `ppo_epoch` y `sub_batch_size`).  
# 5. Medir `avg_train_reward` de esa iteración.  
# 6. Correr `eval_eps` episodios en `env_eval` para `avg_eval_reward`.  
# 7. Actualizar la barra de progreso, guardar métricas en `logs`, y llamar a `plot(logs)`.  

# %% [code]
# 6.1) Variables para logs
logs = {
    "train_reward": [],
    "eval_reward":  []
}

num_eval_episodes = 3  # cuántos episodios de evaluación por iteración

# Barra de progreso
pbar = tqdm(range(num_iterations), desc="PPO Entrenando", unit="iter")

for iter_idx in pbar:
    # —————————————————————————————————————————————————
    # 1) RECOLECTAR frames_per_batch pasos (rollout on-policy)
    # —————————————————————————————————————————————————
    # `experience` es un TensorDict con keys:
    #   "pixels", "action", "log_prob", "state_value", "next_pixels", "reward", "done", "truncated", ...
    experience = next(train_collector)

    # 2) CALCULAR VENTAJAS Y RETORNOS (GAE)
    # ClipPPOLoss internamente requiere que le pasemos un td con:
    #  - "advantage": ventaja por paso
    #  - "return": valor objetivo (retorno descontado)
    #  - "state_value": valor que dio la red en cada paso
    #  - "log_prob": log prob original
    #  - "action": acción tomada
    #  - "reward", "done": info del entorno
    #  - "pixels", "next_pixels": transiciones visuales
    #  → GAE rellena "advantage" y "return" en el mismo TensorDict.
    loss_info = gae_module(experience)
    # Ahora `experience` contiene "advantage" y "return"

    # 3) OPTIMIZAR CON ClipPPOLoss
    #    Le pasamos `experience`, `optimizer`, `ppo_epoch`, `sub_batch_size`
    loss_ppo = clip_ppo_loss(
        experience,
        optimizer=optimizer,
        ppo_epoch=ppo_epoch,
        mini_batch_size=sub_batch_size,
    )

    # 4) CALCULAR avg_train_reward para este batch 
    #    (recompensa total promedio sobre cada episodio dentro de este rollout).
    #    TorchRL almacena en "rollout_info": un tensor con shape [num_episodios_en_batch]
    #    con la suma de rewards por episodio.
    train_episode_rewards = experience.get(("rollout_info", "reward_sum")).cpu().numpy()
    avg_train_reward = float(np.mean(train_episode_rewards))
    logs["train_reward"].append(avg_train_reward)

    # —————————————————————————————————————————————————
    # 5) EVALUACIÓN (política determinista)
    # —————————————————————————————————————————————————
    # Para eval no queremos muestreo aleatorio → cambiamos a modo eval
    set_exploration_type(ExplorationType.MODE)  # fuerza argmax en ProbabilisticActor
    eval_episode_rewards = []
    for _ in range(num_eval_episodes):
        td = env_eval.reset()  # td contiene "pixels"
        done = False
        ep_rew = 0.0
        while not done:
            # Creamos Tensordict de entrada manualmente
            obs_td = td.clone().to(device)
            # Solo necesitamos "pixels" para la política
            out = actor_module(obs_td)
            action = out.get("action").item()
            td, reward, term, trunc, _ = env_eval.step(action)
            done = bool(term or trunc)
            ep_rew += reward
        eval_episode_rewards.append(ep_rew)
    avg_eval_reward = float(np.mean(eval_episode_rewards))
    logs["eval_reward"].append(avg_eval_reward)

    # → Volvemos a modo entrenamiento (“random” sampling)
    set_exploration_type(ExplorationType.RANDOM)

    # 6) ACTUALIZAR tqdm Y GRAficar
    pbar.set_postfix({
        "avg_train": f"{avg_train_reward:.1f}",
        "avg_eval":  f"{avg_eval_reward:.1f}"
    })
    plot(logs)

# Cerrar barra
pbar.close()

# %% [markdown]
# ## 7. Guardar modelo final
# 
# Si quieres salvar el checkpoint del actor y el crítico, puedes hacer:
# ```python
# save_name = f"ppo_cr_{uuid.uuid4().hex[:6]}.pt"
# checkpoint = {
#     "actor_state_dict": actor_module.state_dict(),
#     "value_state_dict": value_module.state_dict(),
# }
# torch.save(checkpoint, save_name)
# print("Modelo guardado en", save_name)
# ```

# %% [markdown]
# ## 8. Observaciones y consejos finales
#
# 1. **Hyperparámetros**  
#    - `frames_per_batch` ≈ 1024–4096 (depende de tu GPU/CPU).  
#    - `num_iterations` = 200–300 (si quieres entrenar más tiempo, subirlo).  
#    - `ppo_epoch` = 3–6, `sub_batch_size` = 128.  
#    - `gamma` = 0.99, `lam` = 0.95, `clip_eps` = 0.2.  
#    - `learning_rate` = 2 × 10⁻⁴.  
#
# 2. **Funciones de TorchRL**  
#    - `ClipPPOLoss` ya engloba:  
#       • Cálculo del ratio πθ/πθ₀ y el clamp.  
#       • Loss de valor (MSE) con coeficiente `value_loss_coeff`.  
#       • Término de entropía (`entropy_coeff`).  
#       • Normalización de ventajas si `normalize_advantage=True` (por defecto).  
#    - `GAE` crea las llaves `"advantage"` y `"return"` dentro del TensorDict.  
#    - `SyncDataCollector` con `frames_per_batch` devuelve un TensorDict con, entre otras keys:  
#       • `"rollout_info","reward_sum"`: array con la suma de recompensas por episodio.  
#
# 3. **Gráficas**  
#    - Cada iteración `plot(logs)` reconstruye la figura desde cero (con `clear_output(wait=True)` y `display(fig)`), de modo que verás en vivo cómo se actualizan las curvas de “Avg Reward (Train)” y “Avg Reward (Eval)”.  
#
# 4. **Exploración**  
#    - Usamos `set_exploration_type(ExplorationType.RANDOM)` durante el entrenamiento para que `ProbabilisticActor` samplee acciones.  
#    - Para evaluación, cambiamos a `ExplorationType.MODE` para usar la acción `argmax`.  
#
# 5. **Ajustes**  
#    - Si deseas ver el entorno “real” en evaluación, podrías crear un Gym env aparte con `render_mode="human"` y dibujar algunos episodios.  
#    - Para guardarlo periódicamente, inserta un bloque dentro del bucle `if iter_idx % 10 == 0: torch.save(...)`.  
#
# ———————————————————————————————————————————————————————————————————————————————
#
# ¡Con esto tienes un pipeline PPO 100 % TorchRL para CarRacing-v3, con gráficos en vivo y barra de progreso!
